In [40]:
import pandas as pd


In [41]:
df = pd.read_csv('../data/csv/tm.csv')
df

,ID,target,split,Hchain_sequence,Lchain_sequence
0,ADI-38502,66.0,train,EVQLLESGGGLVKPGGSLRLSCAASGFIFSDYSMNWVRQAPGKGLE...,DIVMTQSPSTLSASVGDRVTITCRASQSISSWLAWYQQKPGKAPKL...
1,ADI-38501,64.5,train,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYSMNWVRQAPGKGLE...,DIVMTQSPATLSLSPGERATLSCRASQSISTYLAWYQQKPGQAPRL...
2,ADI-47173,64.5,train,EVQLVESGGGVVQPGRSLRLSCAASGFTFDRYGMHWIRQAPGKGLE...,EIVMTQSPLSLPVTLGQPASISCRSNQSLVHSDGNTFLNWFHQRPG...
3,ADI-47054,71.0,train,EVQLVESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLD...,QPVLTQPPSVSVSPGQTARITCSGDALPKQFVYWYQQTPRQAPVLV...
4,ADI-47278,71.5,train,QVQLVQSGAEVKKPGASVKVSCKASGYSFSSYGMHWVRQAPGQRLE...,EIVMTQSPATLSVSPGERATLSCRASQSVSTNLAWYQQKPGQAPRL...
...,...,...,...,...,...
478,urelumab,66.0,holdout,QVQLQQWGAGLLKPSETLSLTCAVYGGSFSGYYWSWIRQSPEKGLE...,EIVLTQSPATLSLSPGERATLSCRASQSVSSYLAWYQQKPGQAPRL...
479,veltuzumab,70.0,holdout,QVQLQQSGAEVKKPGSSVKVSCKASGYTFTSYNMHWVKQAPGQGLE...,DIQLTQSPSSLSASVGDRVTMTCRASSSVSYIHWFQQKPGKAPKPW...
480,visilizumab,71.0,train,QVQLVQSGAEVKKPGASVKVSCKASGYTFISYTMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCSASSSVSYMNWYQQKPGKAPKRL...
481,zalutumumab,72.5,train,QVQLVESGGGVVQPGRSLRLSCAASGFTFSTYGMHWVRQAPGKGLE...,AIQLTQSPSSLSASVGDRVTITCRASQDISSALVWYQQKPGKAPKL...


## load pretrained model for embedding

In [42]:
def embed_esm_batch(batch_sequences, model, alphabet, pooling= False, repr_layer='last'):
    batch_converter = alphabet.get_batch_converter()
    data = [("protein" + str(i), seq) for i, seq in enumerate(batch_sequences)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[model.num_layers], return_contacts=False)

    # Get the embeddings from the last layer
    last_layer = model.num_layers
    token_embeddings = results["representations"][last_layer]

   # If pooling is enabled, return mean-pooled embeddings for each protein
    if pooling:
        sequence_representations_list = []
        for i, tokens_len in enumerate(batch_lens):
            # Pool by averaging over the tokens (excluding padding and special tokens)
            sequence_representation = token_embeddings[i, 1:tokens_len - 1].mean(0)  # Exclude [CLS] and [SEP]
            sequence_representations_list.append(sequence_representation)
        # Return the pooled protein-level embeddings
        return torch.stack(sequence_representations_list)

    # Otherwise, return the full token-level embeddings (excluding special tokens)
    return token_embeddings[:, 1:-1, :]

def embed_protbert_batch(sequences, model, tokenizer, device='cuda' ):
    model.eval()

    sequences_w_spaces = [' '.join(list(seq)) for seq in sequences]
    processed_sequences = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_w_spaces]

    ids = tokenizer.batch_encode_plus(processed_sequences, add_special_tokens=True, pad_to_max_length=True)
    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)

    with torch.no_grad():
        embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]

    return embedding[:,1:-1,:]

In [43]:
### note: loading the ESM 15 Billion parameters takes longer
import torch
import esm

# Load ESM-2 model
# esm.pretrained.esm2_t48_15B_UR50D() 
# esm.pretrained.esm2_t36_3B_UR50D() 
# esm.pretrained.esm2_t33_650M_UR50D()

#model, alphabet = esm.pretrained.esm2_t48_15B_UR50D()
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()
# model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t36_3B_UR50D.pt" to /users/eleves-b/2023/ly-an.chhay/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D.pt


OSError: [Errno 122] Disk quota exceeded

In [39]:
sequence_representations_list = []
save_list = []
chunk_size = 25
for i in range(0, len(df), chunk_size):
    H_batch = df[i:i+chunk_size]["Hchain_sequence"].tolist()
    L_batch = df[i:i+chunk_size]["Lchain_sequence"].tolist()
    ID_batch = df[i:i+chunk_size]["ID"].tolist()
    target_batch = df[i:i+chunk_size]["target"].tolist()
    split_batch = df[i:i+chunk_size]["split"].tolist()
    
    print(i+chunk_size)
    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        H_out = embed_esm_batch(H_batch,model, alphabet,True)
        L_out = embed_esm_batch(L_batch,model, alphabet,True)

        sequence_representations = (H_out + L_out) / 2 
     
    for ID, target,split, embedding in zip(ID_batch, target_batch,split_batch, sequence_representations):

        embedding_list = embedding.cpu().numpy().flatten().tolist()
        
        save_list.append({
            "ID": ID,
            "target": target,
            "split": split,
            "embedding": embedding_list
        })

# Convert to DataFrame
df_save = pd.DataFrame(save_list)

# Save to CSV, embedding as a string
# df_save.to_csv("tm_esm650M.csv", index=False)
df_save.to_csv("tm_esm3B.csv", index=False)
# df_save.to_csv("tm_esm650M.csv", index=False)


25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
